In [1]:
import os
import io
import json
import pandas as pd
from dotenv import load_dotenv
import PyPDF2
import time
from azure.storage.blob import ContainerClient
from openai import AzureOpenAI
from concurrent.futures import ThreadPoolExecutor, as_completed


# Load environment variables
load_dotenv()


# Set up Azure Blob Storage connection
connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
container_name = os.getenv("CONTAINER_NAME")  # Replace with your container name
container_client = ContainerClient.from_connection_string(
    conn_str=connection_string,
    container_name=container_name
)


# Function to extract text from a PDF file stream
def extract_text_from_pdf(file_stream):
    text = ""
    try:
        pdf_reader = PyPDF2.PdfReader(file_stream)
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
    return text


def get_json(file_stream):

    earning_call_transcript = extract_text_from_pdf(file_stream)

    client = AzureOpenAI(
      azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
      api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
      api_version="2024-05-01-preview"
    )

    with open("prompt.txt", 'r') as prompt:
        prompt = prompt.read()
    
    messages = [
        {"role": "system", "content": "Please provide output in a JSON format."},
        {"role": "user", "content": prompt.format(earning_call_transcript=earning_call_transcript)}
    ]


    retries = 1
    while retries <= 3:
        try:
            # print("getting json...")
            response = client.chat.completions.create(
                model = os.getenv("DEPLOYMENT_NAME"),
                messages = messages
            )
            response = response.choices[0].message.content
            print(f"done - {response}")
            return response # how_respose.text
        except Exception as e:
            retries += 1
            if retries > 3:
                print("!!!!!Couldn't Resolve!!!!!!")
                return None
            print(f"Error Occoured: {e}\n Retrying...{retries}")
            time.sleep(15)

def process_blob(blob, k=1, retries=2,):
    attempt = 0
    while attempt < retries:
        try:
            stream = io.BytesIO()
            print(f"File_NAME = {blob.name}")
            container_client.download_blob(blob).readinto(stream)
            op = get_json(stream)
            if op:
                op = json.loads(op[op.index("{"):len(op)-op[::-1].index("}")])
                if op.get("capex"):  # Check if capex is not empty
                    return op
                else:
                    print(f"Capex is empty. Retrying for blob {blob.name}...")
            attempt += 1
            time.sleep(5)  # Wait before retrying
        except Exception as e:
            print(f"Error processing blob {blob.name}: {e}")
            return None
    print(f"Max retries reached for blob {blob.name}.")
    return None

def get_capex_info(companies):
    final_json = []
    blob_list = list(container_client.list_blobs())[:] #limit here
    blob_list = [blob for blob in blob_list if blob.name.strip().split('/')[0] in companies]
    print(f"NUMBER OF FILES: {len(blob_list)}")
    completed = 0
    # Using ThreadPoolExecutor for parallel processing
    with ThreadPoolExecutor(max_workers=1) as executor:
        future_to_blob = {executor.submit(process_blob, blob): blob for blob in blob_list}
        # Iterate over completed futures
        for future in as_completed(future_to_blob):
            blob = future_to_blob[future]
            try:
                result = future.result()
                if result:
                    op = result
                    if op:  # Ensure op is not None
                        completed += 1
                        print(f"{completed} files completed.")
                        print(op, "\n\n") #, how)
                        final_json.append(op)
            except Exception as e:
                print(f"Error processing result for blob {blob.name}: {e}")

    # Ensure final_json contains valid data
    final_json = [item for item in final_json if isinstance(item, dict)]
    if final_json:  # Check if final_json is not empty
        df = pd.DataFrame(final_json)
        print(f"Total {completed} files completed.")
        print(f"Processing completed. Results saved to df.")
        return df, final_json
    else:
        print("No valid data to save to df.")
        return None, None
    


    

    

# Output Preprocess

In [2]:
def clean_df(df):
    df['company'] = df['company'].replace({
    'Amazon': 'Amazon.com',
    'Amazon.com, Inc.': 'Amazon.com',
    'Alphabet':'Alphabet Inc.',
    'Alphabet Inc':'Alphabet Inc.',
    'Amazon.com Inc.': 'Amazon.com',
    'Goldman Sachs' : 'Goldman Sachs Group, Inc.',
    'The Goldman Sachs Group, Inc.': 'Goldman Sachs Group, Inc.',
    'Goldman Sachs Group':'Goldman Sachs Group, Inc.',
    'Tesla' : 'Tesla Inc.',
    'Tesla, Inc.':'Tesla Inc.',
    'Tesla Motors, Inc.': 'Tesla Inc.',
    'Tesla Motors':'Tesla Inc.',
    'Valero Energy Corporation': 'Valero Energy',
    'Cardinal Health':'Cardinal Health, Inc.',
    'BERKSHIRE HATHAWAY INC.': 'Berkshire Hathaway Inc.',
    'Berkshire Hathaway Inc' : 'Berkshire Hathaway Inc.',
    '"Cardinal Health, Inc."':'Cardinal Health, Inc.',
    'Bank of America Corporation': 'Bank of America',
    'CVS Health Corp': 'CVS Health Corporation',
    'CVS Health':'CVS Health Corporation',
    'Cigna Corporation':'CIGNA', 
    'Cigna Corp.':'CIGNA', 
    'Cigna Corp':'CIGNA',
      'CI':'CIGNA',
      'CIGNA Corporation':'CIGNA',
      'Cigna':'CIGNA', 
      'Cigna Group':'CIGNA',
      'The Cigna Group':'CIGNA', 
      'Citigroup Inc.':'Citi',
      'Citigroup Inc':'Citi',
      'CMCSA':'Comcast',
      'Comcast Corporation':'Comcast',
      'Comcast Corp':'Comcast',
      'WellPoint':'Wellpoint Inc',
      'WellPoint':'Wellpoint Inc.',
      'Anthem Inc': 'Anthem Inc.', 
    'Elevance Health':'Elevance Health Inc', 
    'Exxon Mobil Corporation':'ExxonMobil',
      'ExxonMobil Corporation':'ExxonMobil',
       'Ford Motor Company':'Ford Motor', 
       'Ford Motor Co':'Ford Motor',
       'Ford Motor Co.':'Ford Motor',
        'Humana Inc':'Humana Inc.',
       'JPMorgan Chase & Co':'JPMorgan Chase & Co.',
       'JPMorgan Chase':'JPMorgan Chase & Co.',
       'McKesson Corporation':'McKesson' , 
       'McKESSON':'McKesson',
       'McKESSON CORPORATION':'McKesson',  
       'McKesson Corp.':'McKesson',
       'McKesson Corp':'McKesson',
       'Walgreens Boots Alliance':'Walgreens Boots Alliance Inc.',
       'Walgreens Boots Alliance Inc':'Walgreens Boots Alliance Inc.', 
       'Wal-Mart Stores, Inc.':'Walmart', 
      'WAL-MART STORES, INC':'Walmart',
       'Walmart Inc.':'Walmart', 
       'WAL-MART STORES, INC.':'Walmart',
       'WAL-MART STORES, INC. (NYSE: WMT)':'Walmart', 
       'Walmart, Inc.':'Walmart',
      
})

    df['capex'] = df['capex'].replace({ 'Not provided in the report':'-', 'nan':'-' ,'Not provided': '-', 'not available':'-','To be determined':'-','[Data Not Provided]':'-','Not Provided': '-','Not mentioned in the report':'-','X':'-', 'Not available': '-',  'Not specified': '-', 'n/a': '-', 'nan': '-','X.XX': '-','Not provided in the Report':'-','Not specified in the provided document':'-','Not Provided in the Report':'-','Not available in the provided document':'-','Not Available' :'-','[figures not provided]':'-', 'Not specified in the text provided': '-','Not mentioned' : '-','Not disclosed':'-'})
    df['company'] = df['company'].str.replace('"', ' ', regex=False)
    df['year'] = df['year'].replace({'FY16' :'2016', 'FY21':'2021', 'FY20':'2020', 'FY19':'2019', 'FY23':'2023', 'FY24':'2024', 'FY2024':'2024', 'FY17':'2017', 'Q3 FY15':'2015', 'Q1 FY18' : '2018', 'Q1 FY21':'2021'})
    df['quarter'] = df['quarter'].replace({'First Quarter': 'Q1', '2': 'Q2', 'Second Quarter': 'Q2', 'Third Quarter': 'Q3', '3rd Quarter':'Q3', '1':'Q1', '4th':'Q4', '2nd Quarter':'Q2', '2Q':'Q2', '3Q':'Q3', 'Fourth Quarter' : 'Q4','Fourth':'Q4','fourth':'Q4', 'Second':'Q2','4': 'Q4','First':'Q1', '4Q':'Q4','4':'Q4', 'Third':'Q3', '1Q':'Q1'})
    
    # Save the rearranged DataFrame to a new CSV file
    print(f"cleaned data saved")# to {output_file}.")
    return df

def rearrange_df(df):
    # Convert 'quarter' and 'year' columns into a single 'Quarter-Year' column with an underscore
    df['Quarter-Year'] = df['quarter'] + "_" + df['year'].astype(str)

    # Pivot the DataFrame
    pivot_df = df.pivot_table(index='company', columns='Quarter-Year', values='capex', aggfunc='first')

    # Reset the index to make 'company' a column instead of an index
    pivot_df.reset_index(inplace=True)

    # Define a function to sort columns in the desired order
    def sort_columns(df):
        # Extract the current columns
        columns = df.columns.tolist()

        # Extract the company column and the Quarter-Year columns
        company_col = columns[0]
        quarter_cols = columns[1:]

        # Generate sorted columns list: Start with the most recent quarters
        sorted_quarters = sorted(quarter_cols, key=lambda x: (x.split('_')[1], x.split('_')[0]), reverse=True)

        # Combine sorted columns with the company column
        sorted_columns = [company_col] + sorted_quarters
        return sorted_columns

    # Reorder the columns
    sorted_columns = sort_columns(pivot_df)
    pivot_df = pivot_df[sorted_columns]
    pivot_df['company'].unique()
    pivot_df['company'] = pivot_df['company'].str.replace('"', ' ', regex=False)

    print(f"Rearranged data saved")# to {output_file}.")
    return pivot_df

def final_process(df):
    # Define a function to compare capex values between consecutive columns
    def compare_columns(row):
        comparisons = []
        for i in range(1, len(row) - 1):  # Skip the 'company' column
            current_value = row[i]
            next_value = row[i + 1]

            if pd.isna(current_value) or pd.isna(next_value):
                comparisons.append("DNA")#("Data not available")
            else:
                try:
                    current_value = float(current_value)
                    next_value = float(next_value)
                    if next_value < current_value:
                        comparisons.append("Increase")
                    elif next_value > current_value:
                        comparisons.append("Decrease")
                    else:
                        comparisons.append("Unchanged")
                except ValueError:
                    comparisons.append("DNA")#("Data not available")

        return comparisons

    # Apply the comparison function to each row
    comparison_results = df.apply(lambda row: compare_columns(row), axis=1)

    # Replace the original values in the DataFrame with the comparison results
    for i, col in enumerate(df.columns[1:-1]):  # Exclude 'company' and the last quarter
        df[col] = comparison_results.apply(lambda x: x[i])

    # # Save the comparison results to a new CSV file
    # output_file = "capex_comparison_results.csv"
    # df.to_csv(output_file, index=False)

    print(f"Capex comparison results saved") #to {output_file}.")
    return df

# Run and Capture

In [3]:
company_df = []
raw_company_df = []
raw_json = []

In [4]:
# all_companies = [
# "Alphabet",
# "Amazon",
# "Berkshire Hathaway",
# "Cardinal Health",
# "Centene",
# "Chevron",
# "Comcast",
# "ExxonMobil",
# "Tesla",
# "UnitedHealth Group",
# "Valero Energy",
# "Walgreens Boots Alliance",
# "Wallmart"
# ]
all_companies = ["Alphabet"]
for company in all_companies:
    print(f"{'#'*10}processing--{company}{'#'*10}")
    csv_df, json_data = get_capex_info([company])
    raw_json.append(json_data)
    cleaned_df = clean_df(csv_df)
    rearranged_df = rearrange_df(cleaned_df)
    rearranged_df.to_csv(f"{company}.csv")
    raw_company_df.append(rearranged_df.copy(deep=True))
    final_df = final_process(rearranged_df)
    final_df.to_csv(f"final_{company}.csv")
    company_df.append(final_df.copy(deep=True))
    print(f"{'$'*10}processed--{company}{'$'*10}")

final_analysis = pd.concat(company_df)
final_analysis.to_csv("final_analysis_final_test_gpt_aplha.csv")
raw_company_analysis = pd.concat(raw_company_df)
raw_company_analysis.to_csv("final_raw_company_analysis__final_test_gpt_aplha.csv")

##########processing--Alphabet##########
NUMBER OF FILES: 14
File_NAME = Alphabet/2021/2021q1-alphabet-earnings-release.pdf
done - {
  "company": "Alphabet",
  "quarter": "Q1",
  "year": "2021",
  "capex": "5.942"
}
File_NAME = Alphabet/2021/2021q2-alphabet-earnings-release.pdf
1 files completed.
{'company': 'Alphabet', 'quarter': 'Q1', 'year': '2021', 'capex': '5.942'} 


done - {
  "company": "Alphabet Inc.",
  "quarter": "Q2",
  "year": "2021",
  "capex": "11.438"
}
File_NAME = Alphabet/2021/2021q3-alphabet-earnings-release.pdf
2 files completed.
{'company': 'Alphabet Inc.', 'quarter': 'Q2', 'year': '2021', 'capex': '11.438'} 


done - {
  "company": "Alphabet Inc.",
  "quarter": "Q3",
  "year": "2021",
  "capex": "6.819"
}
File_NAME = Alphabet/2021/2021q4-alphabet-earnings-release.pdf
3 files completed.
{'company': 'Alphabet Inc.', 'quarter': 'Q3', 'year': '2021', 'capex': '6.819'} 


done - {
  "company": "Alphabet",
  "quarter": "Q4",
  "year": "2021",
  "capex": "N/A"
}
File_NAM

C:\Users\2272655\AppData\Local\Temp\ipykernel_31200\1720823673.py:113: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_value = row[i]
C:\Users\2272655\AppData\Local\Temp\ipykernel_31200\1720823673.py:114: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  next_value = row[i + 1]


In [5]:
final_analysis

Quarter-Year,company,Q2_2024,Q1_2024,Q4_2023,Q3_2023,Q2_2023,Q1_2023,Q4_2022,Q3_2022,Q2_2022,Q1_2022,Q4_2021,Q3_2021,Q2_2021,Q1_2021
0,Alphabet Inc.,DNA,DNA,DNA,DNA,DNA,DNA,DNA,DNA,DNA,DNA,DNA,Decrease,Increase,5.942


In [6]:
raw_company_analysis

Quarter-Year,company,Q2_2024,Q1_2024,Q4_2023,Q3_2023,Q2_2023,Q1_2023,Q4_2022,Q3_2022,Q2_2022,Q1_2022,Q4_2021,Q3_2021,Q2_2021,Q1_2021
0,Alphabet Inc.,13.186,"12,012","32,251","21,232",13.177,N/A,"31,485","23,890",6.828,"Considered investments in Capex, R&D, and talent",N/A,6.819,11.438,5.942
